In [ ]:
import pdfplumber
import re
import string
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import json

def extract_text_from_pdf(pdf_path):
    text = ""
    with pdfplumber.open(pdf_path) as pdf:
        for page in pdf.pages:
            text += page.extract_text() + "\n"
    return text

def preprocess_text(text):
    text = text.replace("\n", " ").replace("\t", " ").strip()
    return " ".join(text.split())

def prepare_data(pdf_paths, output_file):
    combined_text = ""
    for pdf_path in pdf_paths:
        raw_text = extract_text_from_pdf(pdf_path)
        cleaned_text = preprocess_text(raw_text)
        combined_text += cleaned_text + " "

    combined_text = combined_text.strip()

    with open(output_file, "w", encoding="utf-8") as f:
        f.write(combined_text)
    return combined_text


def chunk_text(text, chunk_size=1500, overlap_size=100):
    chunks = []
    for i in range(0, len(text), chunk_size - overlap_size):
        chunks.append(text[i:i + chunk_size])
    return chunks

def generate_embeddings(texts):
    model = SentenceTransformer('all-MiniLM-L6-v2')

    embeddings = model.encode(texts)
    model.save('all-MiniLM-L6-v2')
    return embeddings

def create_knowledge_base(texts, embeddings):
    knowledge_base = {}
    for i, text in enumerate(texts):
        knowledge_base[i]= {"text": text, "embedding": embeddings[i].tolist() }
    return knowledge_base
def find_relevant_answer(query, knowledge_base):
    model = SentenceTransformer('all-MiniLM-L6-v2')
    query_embedding = model.encode([query])
    scores = {}
    for url, item in knowledge_base.items():
      if item.get('embedding'):
        similarity_score = cosine_similarity(query_embedding, [item['embedding']])[0][0]
        scores[url] = similarity_score
    if not scores:
        return "Sorry, I cannot find an answer for your query."
    best_url = max(scores, key=scores.get)
    if scores[best_url] > 0.3:
      return knowledge_base[best_url]["text"]
    else:
        return "Sorry, I cannot find an answer for your query."

if __name__ == "__main__":
    # pdf_paths = ["/content/textbook1.pdf", "/content/textbook2.pdf", "/content/textbook3.pdf"]
    # output_file = "combined_text.txt"
    # combined_text = prepare_data(pdf_paths, output_file)
    with open('combined_text.txt', 'r') as f:
        combined_text = f.read()
    chunks = chunk_text(combined_text)
    embeddings = generate_embeddings(chunks)
    knowledge_base = create_knowledge_base(chunks,embeddings)
    print("Knowledge base is ready!")

    while True:
        query = input("You: ")
        if query.lower() == "exit":
          break
        response = find_relevant_answer(query, knowledge_base)
        print("Bot: ", response)

In [13]:
!pip install pdfplumber

  Using cached pdfplumber-0.11.5-py3-none-any.whl.metadata (42 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.5/59.5 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 44.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 40.9 MB/s eta 0:00:00


In [21]:
!pip install shutil

ERROR: Could not find a version that satisfies the requirement shutil (from versions: none)
ERROR: No matching distribution found for shutil


In [26]:
import shutil

# Compress the model folder into a zip file
shutil.make_archive('all-MiniLM-L6-v2', 'zip', 'all-MiniLM-L6-v2')


'/content/all-MiniLM-L6-v2.zip'

In [23]:
from google.colab import files

# Download the zipped model
files.download('all-MiniLM-L6-v2.zip')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [28]:
import json
from google.colab import files

# Save the knowledge_base dictionary to a JSON file
with open("knowledge_base.json", "w", encoding="utf-8") as f:
    json.dump(knowledge_base, f, ensure_ascii=False, indent=4)

# Download the JSON file to your local machine
# files.download("knowledge_base.json")
